In [2]:
import json
import requests
import pandas as pd
import numpy as np

In [3]:
base_url = "http://stats.nba.com/stats/leaguedashptstats"
season="2014-15"
season_type="Playoffs"
player_or_team="Player"
measure_type="CatchShoot"
parameters={"Season": season,
            "SeasonType": season_type,
            "PlayerOrTeam": player_or_team,
             "PtMeasureType": measure_type
            }
data=requests.get(base_url,params=parameters)

In [4]:
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/47.0.2526.73 Chrome/47.0.2526.73 Safari/537.36"
REFERER = "http://stats.nba.com/scores/"

In [5]:
def get_sportvu_data_for_stat(season, season_type, player_or_team, measure_type, start_date="", end_date="", last_n_games=0, league_id="00", month=0, opponent_team_id=0, por_round=0, per_mode="Totals", team_id=0, outcome="", location="", season_segment="", vs_conference="", vs_division="", game_scope="", player_experience="", player_position="", starter_bench=""):
    parameters = {
                    "DateFrom": start_date,
                    "DateTo": end_date,
                    "PlayerOrTeam": player_or_team,
                    "PtMeasureType": measure_type,
                    "Season": season,
                    "SeasonType": season_type,
                    "LastNGames": last_n_games,
                    "LeagueID": league_id,
                    "Month": month,
                    "OpponentTeamID": opponent_team_id,
                    "PORound": por_round,
                    "PerMode": per_mode,
                    "TeamID": team_id,
                    "Outcome": outcome,
                    "Location": location,
                    "SeasonSegment": season_segment,
                    "VsConference": vs_conference,
                    "VsDivision": vs_division,
                    "GameScope": game_scope,
                    "PlayerExperience": player_experience,
                    "PlayerPosition": player_position,
                    "StarterBench": starter_bench
    }
    base_url = "http://stats.nba.com/stats/leaguedashptstats"
    return get_data_from_url_with_parameters(base_url, parameters, 0)

In [6]:
def get_data_from_url_with_parameters(base_url, parameters, index):
    measure_type=parameters["PtMeasureType"]
    response = requests.get(base_url, params=parameters, headers={'User-Agent': USER_AGENT, 'referer': REFERER})
    data = response.json()
    headers = data['resultSets'][index]['headers']
    rows = data['resultSets'][index]['rowSet']
    #return [dict(zip(headers, row)) for row in rows]
    df = pd.DataFrame(rows)
    df.columns = headers
    df = df.set_index(['PLAYER_ID', 'TEAM_ABBREVIATION','PLAYER_NAME','TEAM_ID'])
    df.columns = ['{}_{}'.format(measure_type, x) for x in df.columns]
    return df

In [7]:
measures=["CatchShoot", 
"Defense",
"Drives",
"Passing",
"PullUpShot",
"Rebounding",
"Efficiency",
"SpeedDistance",
"ElbowTouch",
"PaintTouch",
"PostTouch",
"Possessions"]

In [8]:
def sportvu_df(season, measure_types,season_type="Regular Season",
player_or_team="Player"):
    sportvu=None
    for m in measure_types:
        print(m)
        input_df=get_sportvu_data_for_stat(season,
        season_type,
        player_or_team,
        measure_type=m)
        if sportvu is None:
            sportvu = input_df
        else:
            sportvu = pd.merge(sportvu,
                               input_df,
                               how='inner',
                               left_index=True,
                               right_index=True)
            print(sportvu.shape)
    sportvu.to_csv('raw_{}'.format(season))
    return sportvu

In [10]:
sportvu_df(season='2014-15',measure_types=measures)
sportvu_df(season='2013-14',measure_types=measures)

CatchShoot
Defense
(492, 22)
Drives
(492, 43)
Passing
(492, 57)
PullUpShot
(492, 69)
Rebounding
(492, 100)
Efficiency
(492, 118)
SpeedDistance
(492, 129)
ElbowTouch
(492, 151)
PaintTouch
(492, 173)
PostTouch
(492, 195)
Possessions
(492, 212)
CatchShoot
Defense
(482, 22)
Drives
(482, 43)
Passing
(482, 57)
PullUpShot
(482, 69)
Rebounding
(482, 100)
Efficiency
(482, 118)
SpeedDistance
(482, 129)
ElbowTouch
(482, 151)
PaintTouch
(482, 173)
PostTouch
(482, 195)
Possessions
(482, 212)


,,,,CatchShoot_GP,CatchShoot_W,CatchShoot_L,CatchShoot_MIN,CatchShoot_CATCH_SHOOT_FGM,CatchShoot_CATCH_SHOOT_FGA,CatchShoot_CATCH_SHOOT_FG_PCT,CatchShoot_CATCH_SHOOT_PTS,CatchShoot_CATCH_SHOOT_FG3M,CatchShoot_CATCH_SHOOT_FG3A,...,Possessions_TIME_OF_POSS,Possessions_AVG_SEC_PER_TOUCH,Possessions_AVG_DRIB_PER_TOUCH,Possessions_PTS_PER_TOUCH,Possessions_ELBOW_TOUCHES,Possessions_POST_TOUCHES,Possessions_PAINT_TOUCHES,Possessions_PTS_PER_ELBOW_TOUCH,Possessions_PTS_PER_POST_TOUCH,Possessions_PTS_PER_PAINT_TOUCH
PLAYER_ID,TEAM_ABBREVIATION,PLAYER_NAME,TEAM_ID,,,,,,,,,,,,,,,,,,,,,
708,BKN,Kevin Garnett,1610612751,53,26,27,1090.0,63,131,0.481,126,0,2,...,45.8,1.19,0.24,0.147,210,137,116,0.143,0.657,0.741
951,MIA,Ray Allen,1610612748,73,47,26,1936.0,124,303,0.409,356,108,270,...,78.1,1.94,1.42,0.290,89,44,40,0.315,0.523,0.525
959,LAL,Steve Nash,1610612747,15,4,11,313.0,7,16,0.438,20,6,13,...,60.5,4.57,5.21,0.128,9,3,3,0.000,0.667,0.667
965,OKC,Derek Fisher,1610612760,81,59,22,1428.0,70,166,0.422,200,60,148,...,122.2,3.37,3.25,0.196,114,11,9,0.184,0.364,0.444
977,LAL,Kobe Bryant,1610612747,6,2,4,177.0,2,10,0.200,5,1,8,...,33.9,4.39,3.77,0.179,31,6,6,0.387,0.000,0.667
979,GSW,Jermaine O'Neal,1610612744,43,26,17,862.0,10,23,0.435,20,0,0,...,29.3,1.68,0.63,0.327,37,128,95,0.514,0.977,1.011
1495,SAS,Tim Duncan,1610612759,74,56,18,2158.0,101,266,0.380,202,0,5,...,123.5,1.65,0.51,0.249,590,533,414,0.298,0.852,0.874
1497,DET,Chauncey Billups,1610612765,19,7,12,309.0,10,28,0.357,28,8,23,...,45.4,4.64,4.18,0.123,18,5,4,0.222,0.000,0.000
1536,LAC,Stephen Jackson,1610612746,9,6,3,107.0,1,13,0.077,3,1,11,...,3.1,1.66,0.68,0.134,4,1,1,0.000,0.000,0.000


In [9]:
sportvu_df(season='2015-16',measure_types=measures)

CatchShoot
Defense
(476, 22)
Drives
(476, 43)
Passing
(476, 57)
PullUpShot
(476, 69)
Rebounding
(476, 100)
Efficiency
(476, 118)
SpeedDistance
(476, 129)
ElbowTouch
(476, 151)
PaintTouch
(476, 173)
PostTouch
(476, 195)
Possessions
(476, 212)


,,,,CatchShoot_GP,CatchShoot_W,CatchShoot_L,CatchShoot_MIN,CatchShoot_CATCH_SHOOT_FGM,CatchShoot_CATCH_SHOOT_FGA,CatchShoot_CATCH_SHOOT_FG_PCT,CatchShoot_CATCH_SHOOT_PTS,CatchShoot_CATCH_SHOOT_FG3M,CatchShoot_CATCH_SHOOT_FG3A,...,Possessions_TIME_OF_POSS,Possessions_AVG_SEC_PER_TOUCH,Possessions_AVG_DRIB_PER_TOUCH,Possessions_PTS_PER_TOUCH,Possessions_ELBOW_TOUCHES,Possessions_POST_TOUCHES,Possessions_PAINT_TOUCHES,Possessions_PTS_PER_ELBOW_TOUCH,Possessions_PTS_PER_POST_TOUCH,Possessions_PTS_PER_PAINT_TOUCH
PLAYER_ID,TEAM_ABBREVIATION,PLAYER_NAME,TEAM_ID,,,,,,,,,,,,,,,,,,,,,
201166,CHI,Aaron Brooks,1610612741,69,36,33,1108.0,35,87,0.402,105,35,85,...,201.5,5.02,5.26,0.204,8,18,5,0.500,0.111,0.400
203932,ORL,Aaron Gordon,1610612753,77,32,45,1857.0,51,158,0.323,142,40,127,...,79.6,1.57,1.02,0.236,84,156,126,0.369,0.821,0.897
1626151,CHA,Aaron Harrison,1610612766,21,15,6,93.0,3,13,0.231,9,3,10,...,3.2,2.15,1.83,0.200,5,2,1,0.200,0.000,0.000
203940,MIN,Adreian Payne,1610612750,52,18,34,486.0,16,47,0.340,39,7,25,...,24.7,1.51,0.59,0.135,74,53,46,0.122,0.453,0.609
201143,ATL,Al Horford,1610612737,82,48,34,2631.0,209,503,0.416,504,86,245,...,125.2,1.43,0.53,0.238,361,398,350,0.211,0.771,0.809
2744,CHA,Al Jefferson,1610612766,47,33,14,1096.0,49,105,0.467,98,0,0,...,46.3,1.87,0.44,0.378,120,256,154,0.208,0.832,1.065
202329,POR,Al-Farouq Aminu,1610612757,82,44,38,2341.0,129,344,0.375,375,117,321,...,91.9,1.80,1.09,0.274,79,86,68,0.316,0.709,0.926
101187,WAS,Alan Anderson,1610612764,13,8,5,192.0,9,28,0.321,26,8,27,...,7.0,1.94,1.00,0.301,3,7,6,0.000,0.714,0.333
1626210,PHX,Alan Williams,1610612756,10,3,7,68.0,0,0,0.000,0,0,0,...,2.4,1.12,0.45,0.225,8,19,16,0.000,0.526,0.500


# dimension reduction and visualization

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn import (manifold,  decomposition, lda, random_projection)

/home/kris/anaconda3/lib/python3.5/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


In [11]:
def clustering(data,year,k_clusters=7):
      center_file=''.join(["nba_centers_",year,".csv"])
      kmeans=KMeans(k_clusters).fit(data)
      centers=kmeans.cluster_centers_
      np.savetxt(center_file,centers)
      return(kmeans)

In [12]:
def dim_reduction(season='2013-14'):
    
    
    file_name_input='_'.join(['raw',season])
    file_name_output='_'.join(['projected',season])
    # reload data
    nba_data=pd.read_csv(file_name_input)
    nba_data=nba_data.set_index(['PLAYER_ID', 'TEAM_ABBREVIATION','PLAYER_NAME','TEAM_ID'])
    
    #nba_data=nba_data.fillna(nba_data.mean(skipna=True))
    #nba_data1=nba_data.convert_objects(convert_numeric=True)
    #nba_data=nba_data[nba_data['CatchShoot_GP']>40]
    #nba_data.shape
    
    player_names=[x[2] for x in nba_data.index]
    team_names=[x[1] for x in nba_data.index]
    
    # data normalized
    scaler=StandardScaler()
    nba_data=scaler.fit_transform(nba_data)
    nba_data=pd.DataFrame(nba_data)
    
    #
    corr = nba_data.T.corr()
    X = corr.as_matrix()

    X_reduced = TruncatedSVD(n_components=10, random_state=0).fit_transform(X)

    method='TSNE'
    year='2013-14'
    if(method=='TSNE'):
        tsne = manifold.TSNE(n_components=2, perplexity=40, verbose=2)
        X_tsne = tsne.fit_transform(X_reduced)
    #      
    kmeans=clustering(data=X_tsne,year=year,k_clusters=7)
    df_projected = pd.DataFrame(X_tsne)
    df_projected['player'] = player_names
    df_projected['team'] = team_names
    df_projected['cluster']=kmeans.labels_[0:df_projected.shape[0]]
    df_projected.columns = ['x', 'y', 'player', 'team','cluster']
    df_projected.to_csv(file_name_output, index=False)

In [13]:
dim_reduction('2013-14')
dim_reduction('2014-15')
dim_reduction('2015-16')

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 482 / 482
[t-SNE] Mean sigma: 2.044452
[t-SNE] Iteration 25: error = 0.7800245, gradient norm = 0.0186138
[t-SNE] Iteration 50: error = 0.8418684, gradient norm = 0.0187316
[t-SNE] Iteration 75: error = 0.5821066, gradient norm = 0.0062818
[t-SNE] Iteration 100: error = 0.6244626, gradient norm = 0.0101878
[t-SNE] KL divergence after 100 iterations with early exaggeration: 0.624463
[t-SNE] Iteration 125: error = 0.5546621, gradient norm = 0.0032595
[t-SNE] Iteration 150: error = 0.6124488, gradient norm = 0.0097579
[t-SNE] Iteration 175: error = 0.6635497, gradient norm = 0.0115150
[t-SNE] Iteration 175: did not make any progress during the last 30 episodes. Finished.
[t-SNE] Error after 175 iterations: 0.624463
[t-SNE] Computing pairwise distances...
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 492 

# Draw the figure using R.  
see 

In [1]:
library(data.table) 
library(scatterD3) 
library(d3heatmap)
library(ggplot2) 
library(ggrepel) 
library(htmlwidgets)

similarity<- function(year){
    file_path=paste('projected_',year,sep='')
    print(file_path)
    dt <- fread(file_path, header=TRUE) # an error arise if read_csv is useddt
    dt=dt[team!='total']
       
    tooltips <- paste("<strong>", dt$player,"</strong><br /><strong>", dt$team, "</strong><br />") 
    p <- scatterD3(x = dt$x, y = dt$y, lab = dt$player, col_var=dt$team, symbol_var=dt$cluters, point_opacity = 0.7, tooltip_text = tooltips, col_lab = "Team", symbol_lab = "Cluster", width=1000, height=1000) 
    saveWidget(p, paste('nba_player_similarity',year,'.html'))
    print(paste(year,"_saved"))
}



In [3]:
years=c('2013-14','2014-15','2015-16')
#year='2015'
for(year in years){
  print(year)
  similarity(year)
}

[1] "2013-14"
[1] "projected_2013-14"
[1] "2013-14 _saved"
[1] "2014-15"
[1] "projected_2014-15"
[1] "2014-15 _saved"
[1] "2015-16"
[1] "projected_2015-16"
[1] "2015-16 _saved"
